In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
from pathlib import Path

sys.path.append(str(Path().cwd().parent))

In [ ]:
from typing import Tuple

import pandas as pd

from plotting import plot_ts
from load_dataset import Dataset
from model import TimeSeriesPredictor

### Какие ряды будем тестировать?

* длинный ряд с сезонностью  
* короткий ряд с сезонностью  
* короткий ряд с сезонностью и трендом  
* случайное блуждание  
* средне зашумленный ряд
* "шумный" ряд  

In [ ]:
ds = Dataset('../data/dataset/')

In [ ]:
long = ds['daily-min-temperatures.csv']

In [ ]:
plot_ts(long)

In [ ]:
short_season = ds['hour_3019.csv'][300:]

In [ ]:
plot_ts(short_season)

In [ ]:
short_season_trend = ds['international-airline-passengers.csv']

In [ ]:
plot_ts(short_season_trend)

In [ ]:
random_walk = ds['dow_jones_0.csv']

In [ ]:
plot_ts(random_walk)

In [ ]:
medium_noize = ds['hour_3426.csv'][300:]

In [ ]:
plot_ts(medium_noize)

In [ ]:
full_noize = ds['day_1574.csv']

In [ ]:
plot_ts(full_noize)

### Какие модели будем тестировать?

* скользящее среднее
* экспоненциальное сглаживание
* autoArima
* линейная регрессия
* линейная регрессия с L1 регуляризацией (Ridge)
* RandomForeset
* градиентный бустинг


In [ ]:
from estimators import RollingEstimator, ExponentialSmoothingEstimator
from pmdarima import auto_arima
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

### По каким метрикам будем сравнивать?

* mse
* mae
* R2
* mape - если не будет ломаться на нулях
* mase

In [ ]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score

from metrics import mean_absolute_percentage_error as mape
from metrics import mase

### По какой методике будем тестировать?

* 70% трейн, 30% тест
* Out-of-sample, чтобы посмотреть как модель предсказывает "вдолгую"
* In-Sample, чтобы посмотреть как модель предсказывает на одну точку вперед
* Для поиска гиперпараметров можно делать кроссвалидацию на тесте по метрике mse

### Задание 1. Напишите функцию, разбивающую на train и test

In [ ]:
def train_test_split(ts: pd.Series, ratio: float = 0.7) -> Tuple[pd.Series]:
    # ваш код здесь
    return ts_train, ts_test

### Зададим соответствие гранулярностей для наших рядов.

In [ ]:
granularity_mapping = {
    'long': 'P1D',
    'short_season': 'PT1H',
    'short_season_trend': 'P1M',
    'random_walk': 'P1D',
    'medium_noize': 'PT1H',
    'full_noize': 'P1D'
}

### Задание 2. Напишите функцию, имплементирующую весь пайплайн обучения и прогноза через TimeSeriesPredictor.

* принмает на вход исходный ряд, гранулярность, количество лагов, модель, а также **kwargs, в которые мы будем передавать параметры модели

* разбивает ряд на train/test

* создает инстанс TimeSeriesPredictor с нужными параметрами

* обучает предиктор на трейне

* делает out_of_sample и in_sample прогноз

* возвращает train, test, in_sample, out_of_sample

In [ ]:
def make_pipeline(ts: pd.Series, granularity: str, num_lags: int, model: callable, **kwargs) -> Tuple[pd.Series]:
    # ваш код здесь
    return train, test, in_sample, out_of_sample

### Задание 3. Напишите функцию, имплементирующую весь пайплайн обучения и прогноза через auto_arima

* функция должна принимать исходный временной ряд, период сезонности, параметры дифференцирования d, D и boolean параметр seasonal, данные параметры будут являться для нас гиперпараметрами, все остальное за нас должна найти auto_arima

* разбивает на train, test

* обучает arima на train при помощи вызова функции auto_arima из библиотеки pmdarima с переданными параметрами и со следующими зафиксированными параметрами: `max_p=3, max_q=3, trace=True, error_action='ignore', suppress_warnings=True, stepwise=True`

* в качестве out_of_sample прогноза просто вызовите метод predict

* в качестве in_sample прогноза обучите модель заново на всём ряде методом `fit`, вызовите метод predict_in_sample и в качестве прогноза возьмите `in_sample_predictions(-len(test):)`

* возвращает train, test, in_sample, out_of_sample (не забудьте сделать их pd.Series с нужным индексом!!)

In [ ]:
def make_pipeline_arima(ts: pd.Series, period: int, d: int = 1, D: int = 1, seasonal: bool = True) -> Tuple[pd.Series]:
    # ваш код здесь
    return train, test, in_sample, out_of_sample

### Задание 4. Напишите функцию, имплементирующую поиск гиперпараметров по сетке. 

* функция должна принимать на вход ряд, гранулярность, модель, дефолтное количество лагов, сетку параметров (словарь)
* после написанного мной кода, функция должна с текущими параметрами запустить пайплайн (функция make_pipeline), получив таким образом прогнозы in_sample и out_of_sample
* посчитать mse для in_sample и out_of_sample прогноза, запомнить их в соответствующие словари
* вернуть лучшие параметры для in_sample и out_of_sample прогнозов

Замечания
* не забудьте, что в сетку параметров можно передавать также num_lags
* если в ряде ts_test есть пропуски, индекс прогноза out_of_sample будет не совпадать c индексом реальных данных, в таком случае, замените индекс out_of_sample прогноза индексом ts_test

In [ ]:
from itertools import product

def hyperparameters_search(ts, granularity, model, num_lags, param_grid, verbose=False):    
    statistics_in_sample, statistics_out_of_sample = {}, {}

    for param_tuple in product(*param_grid.values()):
        params = dict(zip(param_grid.keys(), param_tuple))
        
        # если в сетке есть num_lags, берем его, если нет, берем дефолтный num_lags из сигнатуры функции
        num_lags = params.pop('num_lags', None) or num_lags
        
        <your code here>

### Задание 5. "Прогоните" все алгоритмы на всех рядах и получите сводную таблицу результатов по всем метрикам, постройте также графики прогнозов. 